In [2]:
import os
from pathlib import Path

import numpy as np
import pandas as pd
from ase import Atom, Atoms
from ase.data import chemical_symbols, covalent_radii, vdw_alvarez
from ase.io import read, write
from pymatgen.core import Element
from scipy import stats
from tqdm.auto import tqdm

from mlip_arena.models.utils import REGISTRY, MLIPEnum

model_name = "ORB"

calc = MLIPEnum[model_name].value()

Selected GPU cuda:0 with 40240.80 MB free memory from 1 GPUs


In [14]:
for symbol in tqdm(chemical_symbols[1:]):

    s = set([symbol])

    if "X" in s:
        continue

    try:
        atom = Atom(symbol)
        rmin = min(covalent_radii[atom.number] * 0.5, 1.0)
        rvdw = vdw_alvarez.vdw_radii[atom.number] if atom.number < len(vdw_alvarez.vdw_radii) else np.nan
        rmax = 3.1 * rvdw if not np.isnan(rvdw) else 6
        rstep = 0.01 #if rmin < 1 else 0.4

        a = 2 * rmax

        npts = int((rmax - rmin)/rstep)

        rs = np.linspace(rmin, rmax, npts)
        e = np.zeros_like(rs)

        da = symbol + symbol

        out_dir = Path(REGISTRY[model_name]["family"]) / str(da)

        os.makedirs(out_dir, exist_ok=True)

        skip = 0

        element = Element(symbol)

        try:
            m = element.valence[1]
            if element.valence == (0, 2):
                m = 0
        except:
            m = 0


        r = rs[0]

        positions = [
            [a/2-r/2, a/2, a/2],
            [a/2+r/2, a/2, a/2],
        ]

        traj_fpath = out_dir / f"{model_name}.extxyz"

        if traj_fpath.exists():
            traj = read(traj_fpath, index=":")
            skip = len(traj)
            atoms = traj[-1]
        else:
            # Create the unit cell with two atoms
            atoms = Atoms(
                da,
                positions=positions,
                # magmoms=magmoms,
                cell=[a, a+0.001, a+0.002],
                pbc=True
            )

        print(atoms)

        atoms.calc = calc

        for i, r in enumerate(tqdm(rs)):

            if i < skip:
                continue

            positions = [
                [a/2-r/2, a/2, a/2],
                [a/2+r/2, a/2, a/2],
            ]

            # atoms.set_initial_magnetic_moments(magmoms)

            atoms.set_positions(positions)

            e[i] = atoms.get_potential_energy()

            write(traj_fpath, atoms, append="a")
    except Exception as e:
        print(e)


  0%|          | 0/118 [00:00<?, ?it/s]

Atoms(symbols='H2', pbc=True, cell=[7.4399999999999995, 7.441, 7.441999999999999], calculator=SinglePointCalculator(...))


  0%|          | 0/356 [00:00<?, ?it/s]

Atoms(symbols='He2', pbc=True, cell=[8.866, 8.866999999999999, 8.868], calculator=SinglePointCalculator(...))


  0%|          | 0/429 [00:00<?, ?it/s]

...

Atoms(symbols='Ts2', pbc=True, cell=[12.0, 12.001, 12.002], calculator=SinglePointCalculator(...))


  0%|          | 0/590 [00:00<?, ?it/s]

Atoms(symbols='Og2', pbc=True, cell=[12.0, 12.001, 12.002])


  0%|          | 0/590 [00:00<?, ?it/s]

Class values must be smaller than num_classes.


In [16]:


df = pd.DataFrame(columns=["name", "method", "R", "E", "F", "S^2", "spearman-repulsion", "spearman-attraction"])

for symbol in tqdm(chemical_symbols[1:]):

    da = symbol + symbol

    out_dir = Path(REGISTRY[model_name]["family"]) / da

    traj_fpath = out_dir / f"{model_name}.extxyz"


    if traj_fpath.exists():
        traj = read(traj_fpath, index=":")
    else:
        continue

    Rs, Es, Fs, S2s = [], [], [], []
    for atoms in traj:

        vec = atoms.positions[1] - atoms.positions[0]
        r = np.linalg.norm(vec)
        e = atoms.get_potential_energy()
        f = np.inner(vec/r, atoms.get_forces()[1])
        # s2 = np.mean(np.power(atoms.get_magnetic_moments(), 2))

        Rs.append(r)
        Es.append(e)
        Fs.append(f)
        # S2s.append(s2)

    rs = np.array(Rs)
    es = np.array(Es)
    fs = np.array(Fs)

    indices = np.argsort(rs)[::-1]
    rs = rs[indices]
    es = es[indices]
    fs = fs[indices]

    iminf = np.argmin(fs)
    imine = np.argmin(es)

    data = {
        "name": da,
        "method": model_name,
        "R": Rs,
        "E": Es,
        "F": Fs,
        "S^2": S2s,
        "spearman-descending-force": stats.spearmanr(rs[iminf:], fs[iminf:]).statistic,
        "spearman-ascending-force": stats.spearmanr(rs[:iminf], fs[:iminf]).statistic,
        "spearman-repulsion-energy": stats.spearmanr(rs[imine:], es[imine:]).statistic,
        "spearman-attraction-energy": stats.spearmanr(rs[:imine], es[:imine]).statistic,
    }

    df = pd.concat([df, pd.DataFrame([data])], ignore_index=True)


json_fpath = Path(REGISTRY[model_name]["family"]) / "homonuclear-diatomics.json"

if json_fpath.exists():
    df0 = pd.read_json(json_fpath)
    df = pd.concat([df0, df], ignore_index=True)
    df.drop_duplicates(inplace=True, subset=["name", "method"])

df.to_json(json_fpath, orient="records")

  0%|          | 0/118 [00:00<?, ?it/s]

In [17]:
df

,name,method,R,E,F,S^2,spearman-repulsion,spearman-attraction,spearman-descending-force,spearman-ascending-force,spearman-repulsion-energy,spearman-attraction-energy
0,HH,ORB,"[0.1549999999999998, 0.16504226000000033, 0.17...","[-2.6258459091186523, -2.755453109741211, -1.8...","[0.26894104, -0.0038066, 2.37577677, 7.8109564...",[],NaN,NaN,-0.908097,0.579246,-0.858528,0.504446
1,HeHe,ORB,"[0.13999999999999968, 0.15003038000000046, 0.1...","[-0.37155914306640625, -0.4051551818847656, 0....","[0.02356668, -2.673e-05, 0.45423254, 1.2784304...",[],NaN,NaN,-0.911505,-0.046725,-0.866669,0.340055
2,LiLi,ORB,"[0.6400000000000006, 0.6500202799999997, 0.660...","[8.477561950683594, 8.438533782958984, 8.38090...","[86.12041473, 84.16366577, 82.23960876, 80.216...",[],NaN,NaN,-0.995987,0.999664,-0.999858,0.948886
3,BeBe,ORB,"[0.4800000000000004, 0.49003191999999984, 0.50...","[7.427803039550781, 7.338706970214844, 7.19796...","[116.66287994, 120.23416138, 123.43974304, 126...",[],NaN,NaN,-0.998346,0.972891,-0.958827,0.691532
4,BB,ORB,"[0.41999999999999993, 0.4300200399999996, 0.44...","[1.7283554077148438, 1.9046106338500977, 2.040...","[118.15281677, 121.52095032, 124.86712646, 127...",[],NaN,NaN,-0.970658,0.922968,-0.808702,0.816218
...,...,...,...,...,...,...,...,...,...,...,...,...
112,NhNh,ORB,"[0.09999999999999964, 0.11001697999999926, 0.1...","[-2.2965879440307617, -2.30364990234375, -2.30...","[-0.01143722, -0.01140489, -0.01082459, 0.1164...",[],NaN,NaN,-0.588028,0.989775,-0.215746,0.963378
113,FlFl,ORB,"[0.09999999999999964, 0.11001697999999926, 0.1...","[-3.702641487121582, -3.683434009552002, -3.65...","[-2.14600778, -2.05312586, -1.91521156, 0.2045...",[],NaN,NaN,NaN,-0.539312,-0.457506,0.870783
114,McMc,ORB,"[0.09999999999999964, 0.11001697999999926, 0.1...","[-1.6017036437988281, -1.5843744277954102, -1....","[-0.02472172, 0.00170199, 0.0667266, 0.4929885...",[],NaN,NaN,-0.739683,0.998949,-0.434633,0.999944
115,LvLv,ORB,"[0.09999999999999964, 0.11001697999999926, 0.1...","[-8.688143730163574, -8.684117317199707, -8.66...","[-2.57541442, -2.43290496, -2.19691253, 1.7867...",[],NaN,NaN,NaN,-0.762006,-0.864103,0.903459
